# Notebook Lecture 1: Introduction, Digital Control Systems
© 2025 ETH Zurich, Niclas Scheuer; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook covers the basics of discrete-time control and aliasing.

Authors:
- Niclas Scheuer; nscheuer@ethz.ch

## Import the packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.optimize import curve_fit
from scipy.fftpack import fft, fftfreq
from scipy import signal
from scipy.integrate import solve_ivp

# Example 1: Zero-Order Hold

The zero-order hold is perhaps the simplest method to turn a _discrete-time_ (DT) signal into a _continuous-time_ (CT) signal. It does this by taking the value of the input at time $$t_0 = kT$$ and holding this output for the whole sampling period $$t \in [kT, (k+1)T)$$



In [ ]:
def zero_order_hold_plot(fs):
    T = 2  # Total time in seconds
    t_cont = np.linspace(0, T, 1000)  # Continuous time axis
    signal_cont = np.sin(2 * np.pi * 1 * t_cont)  # Continuous sine wave
    
    t_samples = np.arange(0, T, 1/fs)  # Discrete sample times
    signal_samples = np.sin(2 * np.pi * 1 * t_samples)  # Sampled signal
    
    # Create ZOH signal by repeating each sample value until the next sample
    t_zoh = np.repeat(t_samples, 2)
    signal_zoh = np.repeat(signal_samples, 2)
    
    # Adjust last element to match the original timeline
    t_zoh = np.append(t_zoh, T)
    signal_zoh = np.append(signal_zoh, signal_zoh[-1])
    
    plt.figure(figsize=(8, 4))
    plt.plot(t_cont, signal_cont, 'b', label='Original Signal')  # Continuous signal
    plt.step(t_zoh, signal_zoh, 'r', where='post', label='Zero-Order Hold')  # ZOH signal
    plt.scatter(t_samples, signal_samples, color='black', zorder=3, label='Samples')  # Sample points
    
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(f'Zero-Order Hold (Sampling Frequency = {fs} Hz)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create slider for sampling frequency
fs_slider = widgets.IntSlider(min=1, max=30, step=1, value=5, description='Fs (Hz)')
interactive_plot = widgets.interactive_output(zero_order_hold_plot, {'fs': fs_slider})
display(fs_slider, interactive_plot)


# Example 2: Nyquist-Shannon Sampling Theorem

If we don't sample a signal often enough, we lose important details, making it hard to reconstruct the original signal.

## Sinuisoidal Signals and Sampling
Since any signal can be broken down into sine waves using the Fourier transform, we use a basic sinusoidal signal to explain the Nyquist-Shannon Sampling Theorem.

## Nyquist-Shannon Sampling Theorem
A sampled signal can be perfectly reconstructed only if the sampling rate is at least twice the highest frequency present in the signal.

$$
f_s \geq 2f_{max}
$$

where:
- $f_s$ is the **sampling frequency**
- $f_{max}$ is the **highest frequency component** in the signal





In [ ]:
# Function to generate a sinusoidal signal
def sinusoid(t, f):
    return np.sin(2 * np.pi * f * t)

# Function to fit a sinusoid to sampled points using the lowest frequency possible
def fit_sinusoid(t_samples, y_samples, f_s):
    def model(t, A, f_fit, phase):
        return A * np.sin(2 * np.pi * f_fit * t + phase)
    
    # Nyquist folding to find the aliased frequency
    f_alias = np.abs((f_samples := np.fft.fftfreq(len(t_samples), d=(t_samples[1] - t_samples[0])))[np.argmax(np.abs(np.fft.fft(y_samples)))])
    f_guess = f_alias if f_alias < f_s / 2 else f_s - f_alias
    
    params, _ = curve_fit(model, t_samples, y_samples, p0=[1, f_guess, 0])
    return lambda t: model(t, *params)

# Interactive function to update the plot
def nyquist_plot(f_max, f_s):
    T = 1  # Total duration (1 second)
    t_cont = np.linspace(0, T, 1000)  # High-resolution time for continuous signal
    y_cont = sinusoid(t_cont, f_max)  # Continuous signal

    # Sampling points
    t_samples = np.arange(0, T, 1/f_s)  # Sampled times
    y_samples = sinusoid(t_samples, f_max)  # Sampled values

    # Fit a sinusoid to sampled points
    fitted_sinusoid = fit_sinusoid(t_samples, y_samples, f_s)

    # Generate reconstructed signal
    y_reconstructed = fitted_sinusoid(t_cont)

    # Plot the results
    plt.figure(figsize=(8, 4))
    plt.plot(t_cont, y_cont, 'b', label='Original Signal')  # Continuous signal
    plt.scatter(t_samples, y_samples, color='red', zorder=3, label='Sampled Points')  # Sampled points
    plt.plot(t_cont, y_reconstructed, 'g--', label='Reconstructed Signal')  # Fitted sinusoid
    
    # Nyquist Criterion Check
    nyquist_condition = f_s >= 2 * f_max
    condition_text = "✓ Nyquist Theorem Fulfilled" if nyquist_condition else "✗ Undersampling (Aliasing Can Occur!)"
    plt.title(f'Nyquist-Shannon Sampling Theorem\n{condition_text}', fontsize=12, color='green' if nyquist_condition else 'red')

    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create interactive sliders
f_max_slider = widgets.FloatSlider(min=1, max=10, step=0.5, value=3, description='f_max (Hz)')
f_s_slider = widgets.FloatSlider(min=2, max=20, step=0.5, value=6, description='f_s (Hz)')

# Display interactive plot
interactive_plot = widgets.interactive_output(nyquist_plot, {'f_max': f_max_slider, 'f_s': f_s_slider})
display(f_max_slider, f_s_slider, interactive_plot)


In [ ]:
# Global storage for the original signal
global_t_cont = np.linspace(0, 1, 1000)  # High-resolution time
global_freq_options = np.array([1, 2, 5, 10, 20])  # Available frequency components
global_num_components = 2  # Default number of harmonics
global_f_s = 20  # Default sampling frequency

def generate_signal(num_components):
    global global_freqs, global_amps, global_phases, global_y_cont
    global_freqs = global_freq_options[:num_components]
    global_amps = np.ones(len(global_freqs))
    global_phases = np.random.uniform(0, 2*np.pi, len(global_freqs))
    global_y_cont = sum(A * np.sin(2 * np.pi * f * global_t_cont + p) for A, f, p in zip(global_amps, global_freqs, global_phases))

generate_signal(global_num_components)

# Function to fit multiple sinusoids to sampled points
def fit_sinusoids(t_samples, y_samples, freqs):
    def model(t, *params):
        A = params[:len(freqs)]
        P = params[len(freqs):]
        return sum(A[i] * np.sin(2 * np.pi * freqs[i] * t + P[i]) for i in range(len(freqs)))
    
    p0 = [1] * len(freqs) + [0] * len(freqs)  # Initial guess: Amplitudes = 1, Phases = 0
    params, _ = curve_fit(model, t_samples, y_samples, p0=p0)
    return lambda t: model(t, *params)

# Function to plot Nyquist sampling with complex signals
def nyquist_plot():
    T = 1  # Total duration (1 second)
    num_samples = int(T * global_f_s)  # Adjust number of samples dynamically
    t_samples = np.linspace(0, T, num_samples, endpoint=False)
    y_samples = sum(A * np.sin(2 * np.pi * f * t_samples + p) for A, f, p in zip(global_amps, global_freqs, global_phases))

    # Reconstructed signal
    fitted_sinusoids = fit_sinusoids(t_samples, y_samples, global_freqs)
    y_reconstructed = fitted_sinusoids(global_t_cont)

    # Nyquist Criterion Check
    max_freq = max(global_freqs)
    nyquist_condition = global_f_s >= 2 * max_freq
    condition_text = "\n\n✓ Nyquist Theorem Fulfilled" if nyquist_condition else "\n\n✗ Undersampling (Aliasing Occurs!)"

    # Plot time domain
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.title(f'Nyquist-Shannon Sampling Theorem {condition_text}', fontsize=12, color='green' if nyquist_condition else 'red')
    plt.plot(global_t_cont, global_y_cont, 'b', label='Original Signal')
    plt.scatter(t_samples, y_samples, color='red', zorder=3, label='Sampled Points')
    plt.plot(global_t_cont, y_reconstructed, 'g--', label='Reconstructed Signal')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True)

    # Fourier Transform Analysis
    N = 4096  # Increase FFT size for better resolution
    freqs_cont = fftfreq(N, d=(global_t_cont[1] - global_t_cont[0]))
    spectrum_cont = np.abs(fft(global_y_cont, n=N))
    spectrum_cont /= np.max(spectrum_cont)  # Normalize original spectrum
    
    N_samples = 4096  # Ensure FFT resolution matches
    freqs_samples = fftfreq(N_samples, d=(t_samples[1] - t_samples[0]))
    spectrum_samples = np.abs(fft(y_samples, n=N_samples))
    spectrum_samples /= np.max(spectrum_samples)  # Normalize sampled spectrum
    
    plt.subplot(2, 1, 2)
    plt.plot(freqs_cont[:N//2], spectrum_cont[:N//2], 'b', label='Original Spectrum')
    plt.plot(freqs_samples[:N_samples//2], spectrum_samples[:N_samples//2], 'r--', label='Sampled Spectrum')
    plt.xlim(0, 40)  # Limit frequency axis to 40 Hz
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Normalized Magnitude')
    plt.legend()
    plt.grid(True)

    # Annotate expected peaks
    for f in global_freqs:
        plt.axvline(f, color='gray', linestyle='dashed', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# Interactive controls
num_components_slider = widgets.IntSlider(min=1, max=len(global_freq_options), step=1, value=global_num_components, description='Harmonics')
f_s_slider = widgets.FloatSlider(min=5, max=50, step=1, value=global_f_s, description='f_s (Hz)')

def update_plot(num_components, f_s):
    global global_f_s
    if num_components != len(global_freqs):
        generate_signal(num_components)
    global_f_s = f_s
    nyquist_plot()

interactive_plot = widgets.interactive_output(update_plot, {'num_components': num_components_slider, 'f_s': f_s_slider})
display(num_components_slider, f_s_slider, interactive_plot)


# Exercise 3: Inverted Pendulum Control



In [ ]:
# ---------------------------
# Helper functions
# ---------------------------
def sinusoidal_signal(t, amplitude, frequency):
    """Generate a sinusoidal signal."""
    return amplitude * np.sin(2 * np.pi * frequency * t)

def apply_anti_aliasing_filter(input_signal, cutoff_frequency, sampling_rate=1000):
    """Apply a low-pass Butterworth anti-aliasing filter."""
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = signal.butter(1, normal_cutoff, btype='low', analog=False)
    filtered_signal = signal.filtfilt(b, a, input_signal)
    return filtered_signal

def inverted_pendulum_state_space(m, L, g):
    """
    Convert the inverted pendulum transfer function into a continuous state-space model.
    Given: G(s) = -((3/(2L)) s^2)/(s^2 + (3g)/(2L))
    """
    num = [0, 0, -3/(2*L)]
    den = [1, 0, (3*g)/(2*L)]
    A, B, C, D = signal.tf2ss(num, den)
    return A, B, C, D

# ---------------------------
# Discrete-Time Simulation with ZOH
# ---------------------------
def simulate_and_plot_closed_loop(show_reference_signal, reference_signal_amplitude, reference_signal_frequency, show_error_signal,
                                  noise_amplitude, noise_frequency, 
                                  activate_noise, show_noisy_reference,
                                  activate_aaf, cutoff_frequency, show_reference_aaf,
                                  sampling_frequency, 
                                  pid_p, pid_i, pid_d, 
                                  show_controller_output, show_zero_order_hold, show_sampled_points,
                                  show_pendulum_position):
    with output:
        output.clear_output(wait=True)
        T_total = 20.0               # Total simulation time (s)
        dt_sample = 1.0/sampling_frequency
        t_samples = np.arange(0, T_total+dt_sample, dt_sample)
        
        # Continuous time grid for plotting continuous signals.
        t_cont = np.linspace(0, T_total, 1000)
        # Always compute the reference signal (in degrees) regardless of the visual checkbox.
        r_cont = sinusoidal_signal(t_cont, reference_signal_amplitude, reference_signal_frequency)
            
        if activate_noise:
            noise_cont = sinusoidal_signal(t_cont, noise_amplitude, noise_frequency)
            # If AAF is active, precompute the filtered noise on the continuous grid.
            noise_filt_cont = apply_anti_aliasing_filter(noise_cont, cutoff_frequency, sampling_rate=1000) \
                              if activate_aaf else noise_cont
        else:
            noise_cont = np.zeros_like(t_cont)
            noise_filt_cont = noise_cont

        # Initialize discrete PID controller state variables.
        integral = 0.0
        e_prev = 0.0
        
        # Lists for simulation results.
        t_total_sim = []       # Continuous simulation time.
        x_total_sim = []       # Plant states: [theta (rad), omega].
        u_total_sim = []       # Control signal (held constant over each interval, in radians).
        discrete_times = []    # Sampling instants.
        discrete_u = []        # Discrete PID outputs (radians).
        
        # Lists for discrete error (for Err+Noise with AAF markers).
        discrete_r_list = []       # Reference (degrees) at each sample.
        discrete_theta_list = []   # Plant output (degrees) at each sample.
        discrete_error_AAF_list = []  # (Reference + noise_effect) - plant angle (in degrees)
        
        # Plant parameters.
        m, L, g = 1, 1, 9.81
        A, B, C, D = inverted_pendulum_state_space(m, L, g)
        # Initial plant state: 10° offset (converted to radians) and zero angular rate.
        x_current = np.array([10*np.pi/180, 0])
        
        # Simulation loop over each sampling interval.
        for k in range(len(t_samples)-1):
            t_k = t_samples[k]
            t_next = t_samples[k+1]
            
            # Always sample the reference (in degrees).
            r_sample = sinusoidal_signal(t_k, reference_signal_amplitude, reference_signal_frequency)
            # Store the discrete reference.
            discrete_r_list.append(r_sample)
            
            # Determine the noise sample.
            if activate_noise:
                # For error with AAF, if show_reference_aaf is selected, use filtered noise if available.
                if show_reference_aaf:
                    noise_sample_AAF = np.interp(t_k, t_cont, noise_filt_cont) if activate_aaf else sinusoidal_signal(t_k, noise_amplitude, noise_frequency)
                    # Also, for controller setpoint (for PID), use the "Error with Noise" version.
                    noise_sample = sinusoidal_signal(t_k, noise_amplitude, noise_frequency)
                elif show_noisy_reference:
                    noise_sample = sinusoidal_signal(t_k, noise_amplitude, noise_frequency)
                    noise_sample_AAF = noise_sample  # For consistency.
                else:
                    noise_sample = 0.0
                    noise_sample_AAF = 0.0
            else:
                noise_sample = 0.0
                noise_sample_AAF = 0.0
            
            # The controller "sees" a setpoint.
            # For the PID, we use the reference plus noise if "Show Error with Noise" is selected.
            setpoint_deg = r_sample + (noise_sample if show_noisy_reference else 0.0)
            
            # Sample the plant output (theta in radians) at t_k.
            theta_sample = x_current[0]
            discrete_theta_list.append(np.rad2deg(theta_sample))
            
            # Compute error (in radians) for PID: (setpoint converted to radians) minus plant angle.
            error = np.deg2rad(setpoint_deg) - theta_sample
            
            # Compute discrete PID.
            integral = integral + error * dt_sample
            derivative = 0.0 if k == 0 else (error - e_prev) / dt_sample
            u_discrete = pid_p * error + pid_i * integral + pid_d * derivative  # [radians]
            e_prev = error
            
            # Store the discrete control sample.
            discrete_times.append(t_k)
            discrete_u.append(u_discrete)
            
            # Also compute the discrete error for "Err+Noise with AAF".
            if show_reference_aaf:
                discrete_error_AAF = (r_sample + noise_sample_AAF) - np.rad2deg(theta_sample)
                discrete_error_AAF_list.append(discrete_error_AAF)
            
            # Simulate the continuous plant over this sample period with u_discrete held constant.
            sol = solve_ivp(lambda t, x: A @ x + B.flatten() * u_discrete, 
                            [t_k, t_next], x_current, t_eval=np.linspace(t_k, t_next, 10))
            if k == 0:
                t_total_sim.extend(sol.t.tolist())
                x_total_sim.extend(sol.y.T.tolist())
                u_total_sim.extend([u_discrete]*len(sol.t))
            else:
                t_total_sim.extend(sol.t[1:].tolist())
                x_total_sim.extend(sol.y.T[1:].tolist())
                u_total_sim.extend([u_discrete]*(len(sol.t)-1))
            x_current = sol.y[:, -1]
        
        # Convert simulation results to arrays.
        t_total_sim = np.array(t_total_sim)
        x_total_sim = np.array(x_total_sim)
        u_total_sim = np.array(u_total_sim)
        discrete_times = np.array(discrete_times)
        
        # Convert plant angle from radians to degrees.
        theta_deg = np.rad2deg(x_total_sim[:,0])
        # Convert control signal from radians to degrees.
        u_deg = np.rad2deg(u_total_sim)       # Continuous (ZOH) control signal.
        discrete_u_deg = np.rad2deg(np.array(discrete_u))  # Discrete controller output.
        
        # --- Compute Continuous Error Signals ---
        # Compute reference on the simulation time grid (always computed).
        r_sim = sinusoidal_signal(t_total_sim, reference_signal_amplitude, reference_signal_frequency)
        # Plain error: Reference (without noise) - pendulum angle.
        error_plain = r_sim - theta_deg
        # Error with noise: (Reference + noise) - pendulum angle.
        noise_sim = sinusoidal_signal(t_total_sim, noise_amplitude, noise_frequency) if activate_noise else np.zeros_like(t_total_sim)
        error_noise = (r_sim + noise_sim) - theta_deg
        # Error with AAF: (Reference + noise_effect) - pendulum angle.
        # If AAF is active, filter the noise; otherwise, use unfiltered noise.
        noise_filt_sim = apply_anti_aliasing_filter(noise_sim, cutoff_frequency, sampling_rate=1000) if activate_aaf else noise_sim
        error_aaf = (r_sim + noise_filt_sim) - theta_deg
        
        # --- Plotting ---
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Plot pendulum position.
        if show_pendulum_position:
            ax.plot(t_total_sim, theta_deg, label="Pendulum Angle (°)", color='magenta')
        
        # Plot the continuous reference if desired.
        if show_reference_signal:
            ax.plot(t_cont, r_cont, label="Reference Signal (°)", color='blue', linestyle='--')
        
        # Plot error signals.
        if show_error_signal:
            ax.plot(t_total_sim, error_plain, label="Error (Ref - Pendulum)", color='red', linestyle='-.')
        if show_noisy_reference:
            ax.plot(t_total_sim, error_noise, label="Error with Noise", color='brown', linestyle='--')
        if show_reference_aaf:
            ax.plot(t_total_sim, error_aaf, label="Error with AAF", color='green', linestyle=':')
        
        # Plot controller output.
        if show_controller_output:
            ax.step(t_total_sim, u_deg, 'o', where='post', label="Discrete PID Output (°)", color='black')
        
        # Draw sampled points attached to the error with AAF (if that option is selected).
        if show_sampled_points and show_reference_aaf and discrete_error_AAF_list:
            ax.plot(discrete_times, discrete_error_AAF_list, 'o', label="Sampled Err+Noise with AAF", color='orange')
        
        # Optionally, draw the Zero-Order Hold (ZOH) of the control signal.
        if show_zero_order_hold:
            ax.step(t_total_sim, u_deg, where='post', label="Zero-Order Hold (°)", color='purple', alpha=0.5)
        
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Amplitude (°)")
        ax.set_title("Discrete-Time PID Controller with ZOH on Pendulum Dynamics")
        ax.set_ylim(-20, 20)
        ax.grid(True)
        ax.legend()
        plt.tight_layout()
        plt.show()

# ---------------------------
# Interactive Widgets Setup
# ---------------------------
output = widgets.Output()

# Reference signal section.
reference_signal_amplitude_slider = widgets.FloatSlider(value=5.0, min=0, max=10, step=0.1, description='Ref Amp (°):')
reference_signal_frequency_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description='Ref Freq (Hz):')
show_reference_signal_checkbox = widgets.Checkbox(value=True, description="Show Reference Signal")

# Error section (new) – plots error = (Reference - Pendulum).
show_error_signal_checkbox = widgets.Checkbox(value=True, description="Show Error")

# Noise section.
activate_noise_checkbox = widgets.Checkbox(value=True, description="Activate Noise")
noise_amplitude_slider = widgets.FloatSlider(value=1.0, min=0, max=2, step=0.1, description='Noise Amp (°):')
noise_frequency_slider = widgets.FloatSlider(value=4, min=1, max=10, step=0.1, description='Noise Freq (Hz):')
# Renamed: Show Reference with Noise -> Show Error with Noise.
show_noisy_reference_checkbox = widgets.Checkbox(value=True, description="Show Error with Noise")

# AAF section.
activate_aaf_checkbox = widgets.Checkbox(value=False, description="Activate AAF")
cutoff_frequency_slider = widgets.FloatSlider(value=12.0, min=0.1, max=20, step=0.1, description='Cutoff (Hz):')
# Renamed: Show Ref+Noise with AAF -> Show Err+Noise with AAF.
show_reference_aaf_checkbox = widgets.Checkbox(value=False, description="Show Err+Noise with AAF")

# Sampling section.
sampling_frequency_slider = widgets.FloatSlider(value=5, min=1, max=20, step=1, description='Sampling Freq (Hz):')
show_sampled_points_checkbox = widgets.Checkbox(value=True, description="Show Sampled Points")

# PID section.
pid_p_slider = widgets.FloatSlider(value=1.0, min=0, max=10, step=0.1, description='P Gain:')
pid_i_slider = widgets.FloatSlider(value=0.0, min=0, max=10, step=0.1, description='I Gain:')
pid_d_slider = widgets.FloatSlider(value=0.0, min=0, max=10, step=0.1, description='D Gain:')
show_controller_output_checkbox = widgets.Checkbox(value=True, description="Show Controller Output")

# New ZOH section (moved after PID).
show_zero_order_hold_checkbox = widgets.Checkbox(value=True, description="Show Zero-Order Hold")

# Pendulum position section.
show_pendulum_position_checkbox = widgets.Checkbox(value=True, description="Show Pendulum Position")

# Assemble interactive output.
widgets.interactive_output(simulate_and_plot_closed_loop, {
    'show_reference_signal': show_reference_signal_checkbox,
    'reference_signal_amplitude': reference_signal_amplitude_slider,
    'reference_signal_frequency': reference_signal_frequency_slider,
    'show_error_signal': show_error_signal_checkbox,
    'noise_amplitude': noise_amplitude_slider,
    'noise_frequency': noise_frequency_slider,
    'activate_noise': activate_noise_checkbox,
    'show_noisy_reference': show_noisy_reference_checkbox,
    'activate_aaf': activate_aaf_checkbox,
    'cutoff_frequency': cutoff_frequency_slider,
    'show_reference_aaf': show_reference_aaf_checkbox,
    'sampling_frequency': sampling_frequency_slider,
    'pid_p': pid_p_slider,
    'pid_i': pid_i_slider,
    'pid_d': pid_d_slider,
    'show_controller_output': show_controller_output_checkbox,
    'show_zero_order_hold': show_zero_order_hold_checkbox,
    'show_sampled_points': show_sampled_points_checkbox,
    'show_pendulum_position': show_pendulum_position_checkbox
})

# Layout the control sections.
reference_section = widgets.VBox([show_reference_signal_checkbox, reference_signal_amplitude_slider, reference_signal_frequency_slider])
error_section = widgets.VBox([show_error_signal_checkbox])
noise_section = widgets.VBox([activate_noise_checkbox, noise_amplitude_slider, noise_frequency_slider, show_noisy_reference_checkbox])
aaf_section = widgets.VBox([activate_aaf_checkbox, cutoff_frequency_slider, show_reference_aaf_checkbox])
sampling_section = widgets.VBox([sampling_frequency_slider, show_sampled_points_checkbox])
pid_section = widgets.VBox([pid_p_slider, pid_i_slider, pid_d_slider, show_controller_output_checkbox])
zoh_section = widgets.VBox([show_zero_order_hold_checkbox])
pendulum_section = widgets.VBox([show_pendulum_position_checkbox])

control_layout = widgets.HBox([reference_section, error_section, noise_section, aaf_section, sampling_section, pid_section, zoh_section, pendulum_section])
display(widgets.HBox([control_layout]), output)
